<a href="https://colab.research.google.com/github/CatTaborEP/ProjectMLUTEP/blob/main/MLProject_Take2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Applying Tokenization and Neural Nets

In [16]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
import re
from tqdm import tqdm
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [22]:
#import data as a dataframe
def load_dataset(filename):
  return pd.read_csv(filename)
X = load_dataset('lexile.csv')
#define a label
Y = X['BT Easiness']
excerpt = X['Excerpt']
#drop unecessary columns and label to create a new dataset to work with.
newX =X.drop(['ID','BT s.e.','British WC','Joon\nWC v1','Sentence\nCount v2',
       'Flesch-Reading-Ease', 'Flesch-Kincaid-Grade-Level',
       'Automated Readability Index', 'SMOG Readability',
       'New Dale-Chall Readability Formula', 'CAREC', 'CAREC_M', 'CARES',
       'CML2RI','Last Changed', 'Author','Title','MPAA\nMax','British Words',
       'MPAA \n#Max', 'Anthology','URL','Source','Pub Year','Category','Location','License','Excerpt','BT Easiness','firstPlace_pred', 'secondPlace_pred', 'thirdPlace_pred',
       'fourthPlace_pred', 'fifthPlace_pred', 'sixthPlace_pred','Kaggle split'], axis='columns')
a =newX.columns
print(a)

Index(['MPAA\n#Avg', 'Google\nWC', 'Sentence\nCount v1', 'Paragraphs'], dtype='object')


Preprocess the text of all of the excerpts.  The functions need to have the text all in lower case and use all English words.

In [33]:
def removeContractions(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase

In [34]:
def preprocessing_function(x_train):
    text_list = []
    for text in tqdm(x_train.values):
        text =removeContractions(text)
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = nltk.word_tokenize(text)
        [word for word in text if not word in set(stopwords.words("english"))]
        lemmatizer = nltk.WordNetLemmatizer()
        text = [lemmatizer.lemmatize(word) for word in text]
        text = " ".join(text)
        text_list.append(text)
    text_list = pd.Series(text_list)
    text_list.column = ['Converted_text']
    return text_list

In [35]:
preprocessed_text = preprocessing_function(excerpt)



100%|██████████| 4724/4724 [02:00<00:00, 39.21it/s]


In [36]:
preprocessed_text.head()

0    when the young people returned to the ballroom...
1    all through dinner time mr fayre wa somewhat s...
2    a roger had predicted the snow departed a quic...
3    mr grime wa to come up next morning to sir joh...
4    and outside before the palace a great garden w...
dtype: object

In [37]:
def min_max_scaler(data):
    scaler = MinMaxScaler()
    scaler.fit(data)
    x_scaled = scaler.transform(data)
    return x_scaled

In [38]:
x_scaled= min_max_scaler(newX)

In [39]:
def Tfidfvectorizer(data):
    vectorizer = TfidfVectorizer()
    x_vectorized_train = vectorizer.fit_transform(data)
    return x_vectorized_train

In [40]:
x_vectorized = Tfidfvectorizer(preprocessed_text)
x_vectorized = x_vectorized.toarray()


In [41]:
Z = np.concatenate((x_vectorized, x_scaled), axis = 1)